In [1]:
from dask import do
from distributed import LocalCluster, Executor
from configparser import ConfigParser
import requests
import numpy as np

In [2]:
c = LocalCluster()

In [3]:
e = Executor(c)

In [4]:
def get_latest_temp(location_str, config):
    '''Get latest temperature data from openweather
    params:
        location_str: string with city,country_code
        config: ConfigParser object with openweather section and api_key key
    returns:
        tuple: (location_str, parsed json response)    
    '''
    weather_key = config.get('openweather', 'api_key')
    resp = requests.get('http://api.openweathermap.org/data/2.5/weather',       
                        params={'q': location_str, 
                                'appid': weather_key, 
                                'units': 'metric'}) 
    return location_str, resp.json()
   

In [5]:
def get_forecast(location_str, config):
    '''Get forecast temperature data from openweather
    params:
        location_str: string with city,country_code
        config: ConfigParser object with openweather section and api_key key
    returns:
        tuple: (location_str, parsed json response)
    '''
    weather_key = config.get('openweather', 'api_key')
    resp = requests.get('http://api.openweathermap.org/data/2.5/forecast',       
                        params={'q': location_str, 
                                'appid': weather_key,        
                                'units': 'metric'})
    return location_str, resp.json()

In [6]:
def filter_temp(location_str, weather_json):
    if 'cod' in weather_json.keys() and int(weather_json['cod']) != 200:
        raise ValueError('Bad Data Returned from API: {} - {}'.format(
                location_str, weather_json))
    try:
        api_city_str = '{},{}'.format(weather_json['name'], weather_json['sys']['country'])
    except KeyError:
        api_city_str = '{},{}'.format(weather_json['city']['name'], weather_json['city']['country'])
    resp = {
             'search_city': location_str,
             'api_city': api_city_str,
    }
    if 'main' in weather_json.keys():
        resp['current_temp'] = weather_json['main']['temp']
        resp['current_humidity'] = weather_json['main']['humidity']
    else:
        resp['forecast_temps'] = [fr['main']['temp'] for fr in weather_json['list']]
        resp['forecast_humidity'] = [fr['main']['humidity'] for fr in weather_json['list']]
    return resp

In [7]:
def merge_data(latest, forecast):
    final = latest.copy()
    final.update(forecast)
    mean_tmp, mean_hum = np.mean(forecast['forecast_temps']), np.mean(forecast['forecast_humidity'])
    final['mean_temp'] = np.round(mean_tmp, 2)
    final['mean_hum'] = np.round(mean_hum, 2)
    return final

In [12]:
def main(city, config=get_config()):
    city_str, weather_data = get_latest_temp(city, config)
    latest = filter_temp(city_str, weather_data)
    city_str, weather_data = get_forecast(city, config)
    forecast = filter_temp(city_str, weather_data)
    final = merge_data(latest, forecast)
    return final

In [9]:
def get_config():
    config = ConfigParser()
    config.read('../config/prod.cfg')
    return config

In [10]:
city_list = ['London,UK', 'Berlin,DE', 'NewYork,NY', 
             'LosAngeles,CA', 'Madrid,ES', 'Bangkok,TH', 
             'Baghdad,IQ', 'Auckland,NZ', 'Istanbul,TR',
             'MexicoCity,MX', 'Primavera,CL', 'KualaLumpur,MY',
             'Shanghai,CN', 'Somewhere,WL']

In [11]:
config = get_config()
res = []

In [70]:
%%time

for city in city_list:
    try:
        final = main(city, config)
        res.append(final)
    except Exception as e:
        print(city, e)

print('sorted by current temp: ', sorted(res, key=lambda x: x.get('current_temp'), reverse=True))
print('sorted by upcoming forecast temp: ', sorted(res, key=lambda x: x.get('mean_temp'), reverse=True))

requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org


dict_keys(['dt', 'cod', 'main', 'weather', 'sys', 'coord', 'id', 'base', 'name', 'clouds', 'wind'])
{'cod': '200', 'city': {'sys': {'population': 0}, 'id': 2643743, 'name': 'London', 'country': 'GB', 'population': 0, 'coord': {'lat': 51.50853, 'lon': -0.12574}}, 'list': [{'dt': 1473087600, 'rain': {'3h': 0.0125}, 'dt_txt': '2016-09-05 15:00:00', 'main': {'temp_kf': 1.93, 'grnd_level': 1024.45, 'humidity': 81, 'temp_min': 23.33, 'temp_max': 25.25, 'pressure': 1024.45, 'sea_level': 1032.01, 'temp': 25.25}, 'weather': [{'description': 'light rain', 'icon': '10d', 'id': 500, 'main': 'Rain'}], 'sys': {'pod': 'd'}, 'clouds': {'all': 64}, 'wind': {'deg': 264.002, 'speed': 2.12}}, {'dt': 1473098400, 'rain': {}, 'dt_txt': '2016-09-05 18:00:00', 'main': {'temp_kf': 1.45, 'grnd_level': 1025.42, 'humidity': 77, 'temp_min': 22.97, 'temp_max': 24.42, 'pressure': 1025.42, 'sea_level': 1033.01, 'temp': 24.42}, 'weather': [{'description': 'overcast clouds', 'icon': '04d', 'id': 804, 'main': 'Clouds'}],

requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org


{'cod': '200', 'city': {'sys': {'population': 0}, 'id': 2950159, 'name': 'Berlin', 'country': 'DE', 'population': 0, 'coord': {'lat': 52.524368, 'lon': 13.41053}}, 'list': [{'dt': 1473087600, 'rain': {}, 'dt_txt': '2016-09-05 15:00:00', 'main': {'temp_kf': 2.04, 'grnd_level': 1026.3, 'humidity': 69, 'temp_min': 20.03, 'temp_max': 22.08, 'pressure': 1026.3, 'sea_level': 1032.02, 'temp': 22.08}, 'weather': [{'description': 'clear sky', 'icon': '01d', 'id': 800, 'main': 'Clear'}], 'sys': {'pod': 'd'}, 'clouds': {'all': 0}, 'wind': {'deg': 17.5021, 'speed': 6.27}}, {'dt': 1473098400, 'rain': {}, 'dt_txt': '2016-09-05 18:00:00', 'main': {'temp_kf': 1.53, 'grnd_level': 1029.14, 'humidity': 73, 'temp_min': 16.12, 'temp_max': 17.65, 'pressure': 1029.14, 'sea_level': 1034.88, 'temp': 17.65}, 'weather': [{'description': 'clear sky', 'icon': '01n', 'id': 800, 'main': 'Clear'}], 'sys': {'pod': 'n'}, 'clouds': {'all': 0}, 'wind': {'deg': 16.5023, 'speed': 4.66}}, {'dt': 1473109200, 'rain': {}, 'dt_

requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org


...

dict_keys(['cod', 'id', 'coord', 'name', 'clouds', 'wind', 'dt', 'sys', 'main', 'weather', 'visibility', 'base'])


requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org


{'cod': '200', 'city': {'sys': {'population': 0}, 'id': 3875367, 'name': 'Porvenir', 'country': 'CL', 'population': 0, 'coord': {'lat': -53.299999, 'lon': -70.366669}}, 'list': [{'dt': 1473087600, 'rain': {'3h': 0.0525}, 'dt_txt': '2016-09-05 15:00:00', 'main': {'temp_kf': 3.29, 'grnd_level': 1018.85, 'humidity': 84, 'temp_min': 7.88, 'temp_max': 11.16, 'pressure': 1018.85, 'sea_level': 1029.45, 'temp': 11.16}, 'weather': [{'description': 'light rain', 'icon': '10d', 'id': 500, 'main': 'Rain'}], 'sys': {'pod': 'd'}, 'clouds': {'all': 24}, 'wind': {'deg': 311.002, 'speed': 7.92}}, {'dt': 1473098400, 'rain': {}, 'dt_txt': '2016-09-05 18:00:00', 'main': {'temp_kf': 2.47, 'grnd_level': 1017.96, 'humidity': 76, 'temp_min': 9.37, 'temp_max': 11.84, 'pressure': 1017.96, 'sea_level': 1028.55, 'temp': 11.84}, 'weather': [{'description': 'few clouds', 'icon': '02d', 'id': 801, 'main': 'Clouds'}], 'sys': {'pod': 'd'}, 'clouds': {'all': 24}, 'wind': {'deg': 306.002, 'speed': 8.16}}, {'dt': 1473109

requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org
requests.packages.urllib3.connectionpool - INFO - Starting new HTTP connection (1): api.openweathermap.org


{'cod': '200', 'city': {'sys': {'population': 0}, 'id': 1733046, 'name': 'Kuala Lumpur', 'country': 'MY', 'population': 0, 'coord': {'lat': 3.14309, 'lon': 101.686531}}, 'list': [{'dt': 1473087600, 'rain': {}, 'dt_txt': '2016-09-05 15:00:00', 'main': {'temp_kf': 0.44, 'grnd_level': 1022.26, 'humidity': 95, 'temp_min': 26.68, 'temp_max': 27.11, 'pressure': 1022.26, 'sea_level': 1025.36, 'temp': 27.11}, 'weather': [{'description': 'broken clouds', 'icon': '04n', 'id': 803, 'main': 'Clouds'}], 'sys': {'pod': 'n'}, 'clouds': {'all': 76}, 'wind': {'deg': 323.002, 'speed': 1.42}}, {'dt': 1473098400, 'rain': {'3h': 1.0225}, 'dt_txt': '2016-09-05 18:00:00', 'main': {'temp_kf': 0.41, 'grnd_level': 1022.01, 'humidity': 100, 'temp_min': 24.97, 'temp_max': 25.39, 'pressure': 1022.01, 'sea_level': 1025.15, 'temp': 25.39}, 'weather': [{'description': 'light rain', 'icon': '10n', 'id': 500, 'main': 'Rain'}], 'sys': {'pod': 'n'}, 'clouds': {'all': 76}, 'wind': {'deg': 79.0024, 'speed': 1.76}}, {'dt': 

In [26]:
%%time

futures = [e.submit(main, i) for i in city_list]
print(futures)
sorted([f.result() for f in futures if f.status != 'error'], key=lambda x: x['current_temp'], reverse=True)
sorted([f.result() for f in futures if f.status != 'error'], key=lambda x: x['mean_temp'], reverse=True)

[<Future: status: finished, type: dict, key: main-189a8f61630da68066ad8287f070b8ea>, <Future: status: finished, type: dict, key: main-b67bf67cf33a0da835476242b530363b>, <Future: status: finished, type: dict, key: main-e1f4d25735bd44f90a7a79e9c48424b3>, <Future: status: finished, type: dict, key: main-a40de7f26ee7767a3a7471a1cc3e0eb6>, <Future: status: finished, type: dict, key: main-fda51f794c8530d33a9bc10064d37ee3>, <Future: status: finished, type: dict, key: main-170951ae38f1a917c19414a18f01558a>, <Future: status: finished, type: dict, key: main-66ad48082a9197924da6d23f455a68bf>, <Future: status: finished, type: dict, key: main-312676950045989c85956638ec088c8e>, <Future: status: finished, type: dict, key: main-9d76328810ca2a9f1085aa06e17e2e87>, <Future: status: finished, type: dict, key: main-56bc9ef3426adb18d5f39cee3ed91d19>, <Future: status: finished, type: dict, key: main-6e04c55654b10296d888be3994e36b56>, <Future: status: finished, type: dict, key: main-5aa7985f3a2297205f9258b1c1

In [32]:
example_error = futures[-1]

In [33]:
example_error.status

'error'

In [58]:
example_error.result()

ValueError: Bad Data Returned from API: Somewhere,WL - {'message': 'Error: Not found city', 'cod': '404'}